In [2]:
from bs4 import BeautifulSoup

In [3]:
# open the html file with the records and load into the HTML parser
with open('records.html') as f:
    html_text = f.read()
soup = BeautifulSoup(html_text,'html.parser')

### Parse Page

In [4]:
# get the columns
columns = list()
for column_name in soup.find_all('th'):
    columns.append(column_name.text)

In [8]:
class Record():
    def __init__(self, track, cc, time, player, nation, date, duration, character, vehicle,tires, glider, splits, mushroom, coin, hybrid_used):
        self.track = track
        self.cc = cc
        self.time = time
        self.player = player
        self.nation = nation
        self.date = date
        self.duration = duration
        self.character = character
        self.vehicle = vehicle
        self.tires = tires
        self.glider = glider
        self.splits = splits
        self.mushroom = mushroom
        self.coin = coin
        self.hybrid_used = hybrid_used

In [31]:
rows = soup.find_all('tr')
tied_times = {
    'Mario Circuit': 3,
    'GCN Baby Park': 3,
    'Big Blue': 3,
    'Tour Sydney Sprint': 3
}
def number_of_times(course_name):
    if course_name in tied_times: return tied_times[course_name]
    else: return 2
i = 1
while i < len(rows):
    course_name = ''
    time_150 = 0
    hybrid_150 = False
    main_row = rows[i]
    cols = main_row.find_all('td',recursive=False)
    # if cols[0].td is not None:
    #     print(cols[0].td)
    for j, col in enumerate(main_row.find_all('td',recursive=False)):
        if j==0:
            if col.td is not None:
                course_name = col.td.text
        elif j==1:
            time_150 = col.text
            if col.img is not None:
                hybrid_150 = True
        elif j ==2:
            player_150 = col.text
        elif j ==3:
            country_150 = col.img.get_attribute_list('title')[0]
        elif j ==4:
            date_150 = col.text
        elif j ==5:
            duration_150 = col.text
        elif j==6:
            character_150 = col.text
        elif j==7:
            kart_150 = col.text
        elif j==8:
            wheels_150 = col.text
        elif j==9:
            glider_150 = col.text
        elif j==10:
            print(col.img.get_attribute_list('onmouseover'))
        else:
            pass
            print(j,col)
            # print(col.text)
    # print(time_150)
    break
    i += number_of_times(course_name)+2
    # print(i)
    
    

2022-07-06
["show_splits(\n    'ttipid_splits', '33.047', '30.906', '30.759', '8-2-0', '1-1-1');"]


In [6]:
# # len(soup.find_all('tr'))
# soup.find_all('tr')[257]
# for i in range(1,261,4):
#     print(i)
